# Weekly Report for Hospital Data

In [294]:
# Imports
import sys
import pandas as pd
import psycopg
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import numpy as np

from credentials import DB_PASSWORD, DB_USER

In [56]:
# Connect to DB
conn = psycopg.connect(
    host="sculptor.stat.cmu.edu", dbname=DB_USER,
    user=DB_USER, password=DB_PASSWORD
)
cur = conn.cursor()

In [57]:
# Summary I & II
with conn.transaction():
    cur.execute(
        "SELECT report_date AS week, COUNT(*) AS records_loaded, "
        "SUM(total_adult_hospital_beds), SUM(total_pediatric_hospital_beds), "
        "SUM(total_adult_hospital_beds_occupied), "
        "SUM(total_pediatric_hospital_beds_occupied), "
        "SUM(inpatient_beds_occupied_covid) "
        "FROM facility_reports "
        "GROUP BY report_date "
        "ORDER BY week DESC;"
    )

records = pd.DataFrame(
    cur.fetchall(), columns=["Week", "Records Loaded", "Total Adult Hospital Beds Available", 
                             "Total Pediatric Hospital Beds Available", 
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total Hospital Beds Used by COVID cases"]
)

In [58]:
# I
records[["Week", "Records Loaded"]]

,Week,Records Loaded
0,2022-10-21,4985
1,2022-10-14,4995
2,2022-10-07,4996
3,2022-09-30,4997
4,2022-09-23,4996


In [136]:
# I table
import plotly.figure_factory as ff

colorscale1 = [[0, '#379392'],[.5, '#9DC3C1'],[1, '#D8E6E7']]
colorscale2 = [[0, '#eb9f9f'],[.5, '#f1bbba'],[1, '#FADAD8']]
colorscale3 = [[0, '#79bd9a'],[.5, '#a8dba8'],[1, '#cff09e']]
colorscale4 = [[0, '#6a60a9'],[.5, '#dedcee'],[1, '#F8FAFF']]
df_1 = records[["Week", "Records Loaded"]]

fig_1 = ff.create_table(df_1, colorscale=colorscale4)

fig_1.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Summary of loaded records",
        "y":0.95,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=150*df_1.shape[1],
    height=40*df_1.shape[0]
)

fig_1.show()

In [5]:
# II
records

,Week,Records Loaded,Total Adult Hospital Beds Available,Total Pediatric Hospital Beds Available,Total Adult Hospital Beds In-Use,Total Pediatric Hospital Beds In-Use,Total Hospital Beds Used by COVID cases
0,2022-10-21,4985,414880.3,39882.2,469153.4,29706.5,24116.8
1,2022-10-14,4995,412939.0,39711.6,467145.9,28799.0,23695.5
2,2022-10-07,4996,415465.4,39796.8,465980.5,28285.8,23689.3
3,2022-09-30,4997,416279.4,39804.7,462204.5,28042.8,24142.5
4,2022-09-23,4996,413589.5,39605.5,463698.4,27778.1,25600.6


In [178]:
# III
with conn.transaction():
    cur.execute(
        "SELECT facility_name, rating, "
        "total_adult_hospital_beds_occupied, "
        "total_pediatric_hospital_beds_occupied, "
        "total_icu_beds_occupied, "
        "inpatient_beds_occupied_covid "
        "FROM facility_information info "
        "JOIN quality_ratings rate "
        "ON info.facility_id = rate.facility_id "
        "JOIN facility_reports "
        "ON info.facility_id = hospital_pk "
        "ORDER BY rating DESC;"
        
    )

ratings = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Hospital Rating",
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total ICU Beds In-Use", "Total Hospital Beds Used by COVID cases"]
)

In [179]:
ratings

,Hospital Name,Hospital Rating,Total Adult Hospital Beds In-Use,Total Pediatric Hospital Beds In-Use,Total ICU Beds In-Use,Total Hospital Beds Used by COVID cases
0,LBJ TROPICAL MEDICAL CENTER,None,59,10,5.4,0
1,LBJ TROPICAL MEDICAL CENTER,None,59,10,5.4,0
2,BIBB MEDICAL CENTER,None,7.3,0,0,0
3,HALE COUNTY HOSPITAL,None,7.4,0,0,0
4,HALE COUNTY HOSPITAL,None,7.4,0,0,0
...,...,...,...,...,...,...
68806,HCA FLORIDA LAWNWOOD HOSPITAL,1,325.9,20,64.6,12.7
68807,HCA FLORIDA LAWNWOOD HOSPITAL,1,325.9,20,64.6,12.7
68808,HCA FLORIDA LAWNWOOD HOSPITAL,1,325.9,20,64.6,12.7
68809,VICTOR VALLEY GLOBAL MEDICAL CENTER,1,41,0,5.3,None


In [297]:
# III - 2
with conn.transaction():
    cur.execute(
        "SELECT rating, "
        "SUM(total_adult_hospital_beds_occupied + total_pediatric_hospital_beds_occupied + "
        "total_icu_beds_occupied + inpatient_beds_occupied_covid) "
        "/ SUM(total_adult_hospital_beds + total_pediatric_hospital_beds + total_icu_beds) AS fraction, "
        "SUM(total_adult_hospital_beds_occupied + total_pediatric_hospital_beds_occupied + "
        "total_icu_beds_occupied + inpatient_beds_occupied_covid) AS total_used, "
        "SUM(total_adult_hospital_beds + total_pediatric_hospital_beds + total_icu_beds) AS total_beds "
        "FROM facility_information info "
        "JOIN quality_ratings rate "
        "ON info.facility_id = rate.facility_id "
        "JOIN facility_reports "
        "ON info.facility_id = hospital_pk "
        "GROUP BY rating "
        "ORDER BY rating DESC; "
        
    )

ratings_2 = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Rating", "Fraction",
                             "Total Hospital Beds In-Use", "Total Hospital Beds"]
)

In [298]:
ratings_2

,Hospital Rating,Fraction,Total Hospital Beds In-Use,Total Hospital Beds
0,None,0.60801862493292531039,283388.3,466084.9
1,5,1.1253921112248581,804807.4,715135.1
2,4,1.0688170139231836,1721683.9,1610831.3
3,3,0.93335884722623767254,1704251.0,1825933.3
4,2,0.87367514514722097003,1292134.0,1478963.9
5,1,0.95637748823622834536,521489.5,545275.8


In [305]:
# III - 2 - table

df_2_2 = ratings_2

fig_2_2 = ff.create_table(df_2_2, colorscale=colorscale3)

fig_2_2.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Fraction of beds currently in use by hospital quality rating",
        "y":0.96,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=220*df_2_2.shape[1],
    height=40*df_2_2.shape[0]
)

fig_2_2.show()

In [240]:
# IV
with conn.transaction():
    # Extract the number of beds in use    
    cur.execute("SELECT report_date, SUM(total_occupied_0) "
                "AS total_occupied_all, "
                "SUM(inpatient_beds_occupied_covid) "
                "AS total_occupied_covid FROM "
                "(SELECT report_date, inpatient_beds_occupied_covid, "
                "total_adult_hospital_beds_occupied + "
                "total_pediatric_hospital_beds_occupied + "
                "total_icu_beds_occupied + "
                "inpatient_beds_occupied_covid AS total_occupied_0 "
                "FROM facility_reports) AS a "
                "GROUP BY report_date "
                "ORDER BY report_date;")
    data_used = pd.DataFrame(
        cur.fetchall(), columns=["report_date", 
                                 "all_used_beds",
                                 "covid_used_beds"]
        )


# Make the bar plot
fig_4_bar = go.Figure()
fig_4_bar.add_trace(go.Bar(
    x=data_used['report_date'],
    y=data_used['all_used_beds'],
    text=data_used['all_used_beds'],
    textposition='auto',
    name='All cases',
    marker_color='#41D3BD'
))
fig_4_bar.add_trace(go.Bar(
    x=data_used['report_date'],
    y=data_used['covid_used_beds'],
    text=data_used['covid_used_beds'],
    textposition='auto',
    name='COVID cases',
    marker_color='#379392'
))

fig_4_bar.update_layout(
    title={
        ########## Note: need to change the date to input
        "text":"Number of beds in use of all cases and covid cases",
        "y":0.88,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    plot_bgcolor='#D8E6E7')

fig_4_bar.show()

In [292]:
# Make the pie chart
labels = ['All cases', 'COVID cases']

# Define color sets of paintings
colors1 = ['#88dba3', '#cff0da']
colors2 = ['#4ea1d3', '#d8e9ef']
colors3 = ['#6a60a9', '#dedcee']
colors4 = ['#fbd14b', '#fffcf0']
colors5 = ['#ee6e9f', '#fec9c9']

# Create subplots, using 'domain' type for pie charts
specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}], 
        [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
fig_4_pie = make_subplots(rows=2, cols=3, specs=specs)

# Define pie charts
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][0],
                    data_used["covid_used_beds"][0]], name=str(data_used['report_date'][0]),
                     marker_colors=colors1), 1, 1)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][1],
                    data_used["covid_used_beds"][1]], name=str(data_used['report_date'][1]),
                     marker_colors=colors2), 1, 2)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][2],
                    data_used["covid_used_beds"][2]], name=str(data_used['report_date'][2]),
                     marker_colors=colors3), 1, 3)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][3],
                    data_used["covid_used_beds"][3]], name=str(data_used['report_date'][3]),
                     marker_colors=colors4), 2, 1)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][4],
                    data_used["covid_used_beds"][4]], name=str(data_used['report_date'][4]),
                     marker_colors=colors5), 2, 2)

# Tune layout and hover info
fig_4_pie.update_traces(hole=.4, hoverinfo='label+percent+name', textinfo='none')
fig_4_pie.update_layout(
    title={
        "text":"Number of beds in use of all cases and covid cases",
        "y":0.88,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    showlegend=False,
    annotations=[dict(text='9-23', x=0.125, y=0.825, font_size=13, showarrow=False),
                 dict(text='9-30', x=0.875, y=0.825, font_size=13, showarrow=False),
                 dict(text='10-07', x=0.5, y=0.825, font_size=13, showarrow=False),
                 dict(text='10-14', x=0.5, y=0.18, font_size=13, showarrow=False),
                 dict(text='10-21', x=0.12, y=0.18, font_size=13, showarrow=False)]
    )

fig_4_pie = go.Figure(fig_4_pie)
fig_4_pie.show()

In [211]:
# Additional I - COVID map - All weeks
# Make a new SAVEPOINT
with conn.transaction():    
    # Get the data for COVID map in each state
    cur.execute("SELECT state, SUM(total_covid_0) "
                "AS total_covid FROM "
                "(SELECT hospital_pk, adult_icu_patients_confirmed_covid "
                "+ inpatient_beds_occupied_covid AS total_covid_0 "
                "FROM facility_reports) AS a "
                "INNER JOIN "
                "(SELECT facility_id, state "
                "FROM facility_information) AS b "
                "ON a.hospital_pk = b.facility_id "
                "GROUP BY state; ")
    
    #cur.execute("SELECT state, SUM(total_covid_0) AS total_covid FROM (SELECT hospital_pk, adult_icu_patients_confirmed_covid + inpatient_beds_occupied_covid AS total_covid_0 FROM facility_reports) AS a INNER JOIN(SELECT facility_id, state FROM facility_information) AS b ON a.hospital_pk = b.facility_id GROUP BY state;")
    data_map1 = pd.DataFrame(cur.fetchall(), columns=["state", "beds"])

# Make the map
fig_map1 = go.Figure(data=go.Choropleth( 
    locations=data_map1['state'], 
    z = data_map1['beds'].astype(float), # Set fill color
    locationmode = 'USA-states', # Set the country 
    colorscale = 'reds', # Set the legend color
    colorbar_title = "Number", # Legend title
)) 
 
fig_map1.update_layout( 
    title={
        "text": "The number of COVID cases by state",
        "y":0.95,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    geo_scope='usa', # Set country
    autosize=True,
    margin=dict(t=30,b=25,l=0,r=0)
)


fig_map1.show()



In [143]:
# Additional I - COVID map - per week
with conn.transaction():
    # Get the data for COVID map in each state
    cur.execute("SELECT state, report_date, SUM(total_covid_0) "
                "AS total_covid FROM "
                "(SELECT hospital_pk, report_date, "
                "adult_icu_patients_confirmed_covid "
                "+ inpatient_beds_occupied_covid AS total_covid_0 "
                "FROM facility_reports "
                "GROUP BY report_date, hospital_pk) AS a "
                "INNER JOIN "
                "(SELECT facility_id, state "
                "FROM facility_information) AS b "
                "ON a.hospital_pk = b.facility_id "
                "GROUP BY state, report_date "
                "ORDER BY state;")
    
    data_map2 = pd.DataFrame(cur.fetchall(), columns=["state", "week", "beds"])

########## Note: need to change the date to input
data_map_tw = data_map2.iloc[np.where(
    data_map2["week"]==pd.Timestamp("2022-10-21"))[0],:]

# Make the map
fig_map2 = go.Figure(data=go.Choropleth( 
    locations=data_map_tw['state'], 
    z = data_map_tw['beds'].astype(float), # Set fill color
    locationmode = 'USA-states', # Set the country 
    colorscale = 'Reds', # Set the legend color
    colorbar_title = "Number", # Legend title
)) 

fig_map2.update_layout(
    title={
        ########## Note: need to change the date to input
        "text":"The number of COVID cases by state" + ": 2022-10-21",
        "y":0.95,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    geo_scope='usa', # Set country
    autosize=True,
    margin=dict(t=30,b=25,l=0,r=0)
)


/var/folders/vw/0kbg8m6n26sdbzft__pgcwc80000gn/T/ipykernel_15568/3257932304.py:22: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



In [158]:
# Additional - II - State COVID increase
# Note: Change variables!
with conn.transaction():
    cur.execute(
        "SELECT tw.state, tw.covidcase - lw.covidcase AS coviddiff "
        "FROM ("
        # This weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        "GROUP BY state) tw "
        "JOIN ("
        # Last weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        "GROUP BY state) lw "
        "ON tw.state = lw.state "
        "ORDER BY coviddiff DESC;",
        {
            "week": pd.to_datetime("2022-10-21"),
            "lastweek": pd.to_datetime("2022-10-21") - pd.Timedelta(days=7)
        }
    )

statecovid = pd.DataFrame(
    cur.fetchall(), columns=["States", "Change in Covid Case"]
)

In [159]:
statecovid

,States,Change in Covid Case
0,GU,None
1,VI,None
2,IL,133.3
3,WI,81.4
4,OH,79.9
5,IN,67.8
6,MN,65.9
7,AZ,64.3
8,PA,60.4
9,MO,50.1


In [209]:
# Additional - II table
df_a_2 = statecovid

fig_a_2 = ff.create_table(df_a_2, colorscale=colorscale4)

fig_a_2.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"State COVID cases changes",
        "y":0.995,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=240*df_a_2.shape[1],
    height=30*df_a_2.shape[0]
)

fig_a_2.show()

In [160]:
# Additional - III - Top Hospital COVID increase
# Note: Change variables!
with conn.transaction():
    cur.execute(
        "SELECT info.facility_name, tw.covidcase - lw.covidcase AS coviddiff, "
        "address, city, state, zipcode "
        "FROM ("
        # This weeks data
        "SELECT facility_id, inpatient_beds_occupied_covid as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        ") tw "
        "JOIN ("
        # Last weeks data
        "SELECT facility_id, inpatient_beds_occupied_covid as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        ") lw "
        "ON tw.facility_id = lw.facility_id "
        "JOIN facility_information info "
        "ON info.facility_id = tw.facility_id "
        "WHERE tw.covidcase - lw.covidcase IS NOT NULL "
        "ORDER BY coviddiff DESC "
        "LIMIT 5",
        {
            "week": pd.to_datetime("2022-10-21"),
            "lastweek": pd.to_datetime("2022-10-21") - pd.Timedelta(days=7)
        }
    )

hospitalcovid = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Change in Covid Case",
                             "Address", "City", "State", "Zipcode"]
)

In [161]:
hospitalcovid

,Hospital Name,Change in Covid Case,Address,City,State,Zipcode
0,CAROLINAS MEDICAL CENTER/BEHAV HEALTH,29.4,1000 BLYTHE BLVD,CHARLOTTE,NC,28203
1,NEW YORK-PRESBYTERIAN HOSPITAL,24.7,525 EAST 68TH STREET,NEW YORK,NY,10065
2,GEISINGER MEDICAL CENTER,17.7,100 NORTH ACADEMY AVENUE,DANVILLE,PA,17822
3,HOUSTON METHODIST HOSPITAL,16.2,6565 FANNIN,HOUSTON,TX,77030
4,MAINE MEDICAL CENTER,15.8,22 BRAMHALL ST,PORTLAND,ME,4102


In [201]:
# Additional - III table
hospitalcovid_new = hospitalcovid

hospitalcovid_new["Hospital Name"] = [
    "CAROLINAS MEDICAL CENTER<br />/BEHAV HEALTH",
    "NEW YORK-PRESBYTERIAN HOSPITAL",
    "GEISINGER MEDICAL CENTER",
    "HOUSTON METHODIST HOSPITAL",
    "MAINE MEDICAL CENTER"]

df_a_3 = hospitalcovid_new

fig_a_3 = ff.create_table(df_a_3, colorscale=colorscale2)

fig_a_3.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Hospitals with largest changes in COVID cases",
        "y":0.97,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=240*df_a_3.shape[1],
    height=60*df_a_3.shape[0]
)

fig_a_3.show()

In [146]:
# Additional - IV - Hospital with no data
# Note: Change thisweek to the parameter of this week
with conn.transaction():
    cur.execute(
        "SELECT facility_name, MAX(report_date) "
        "FROM facility_information "
        "JOIN facility_reports ON "
        "facility_id = hospital_pk "
        "GROUP BY facility_id "
        "HAVING MAX(report_date) < %(week)s "
        "ORDER BY MAX(report_date) DESC;",
        {"week": pd.to_datetime("2022-10-21")}
    )

nodata = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Last reported Date"]
)

In [147]:
nodata

,Hospital Name,Last reported Date
0,WAGONER COMMUNITY HOSPITAL,2022-10-14
1,DRUMRIGHT REGIONAL HOSPITAL,2022-10-14
2,KNOX COUNTY HOSPITAL,2022-10-14
3,SYRACUSE AREA HEALTH,2022-10-14
4,UNITY MEDICAL CENTER,2022-10-14
5,WINSTON MEDICAL CENTER & SWINGBED,2022-10-14
6,MASSAC MEMORIAL HOSPITAL,2022-10-14
7,HOLY CROSS HOSPITAL,2022-10-14
8,LA PAZ REGIONAL HOSPITAL,2022-10-14
9,MT SINAI HOSPITAL MEDICAL CENTER,2022-10-14


In [172]:
# Additional - IV table
df_a_4 = nodata

fig_a_4 = ff.create_table(df_a_4, colorscale=colorscale1)

fig_a_4.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Hospital with no reported data",
        "y":0.98,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=380*df_a_4.shape[1],
    height=30*df_a_4.shape[0]
)

fig_a_4.show()